In [1]:
# Create IMAM supervision data to upload into RapidPro

# Identify all supervision (State and LGA) and put in rows by site (name, phone, email)
# Merge State and LGA supervision into complete listing of personnel
# export as .xlsx file

# This is still very excel way of thinking. 
# We should just create the table in json and import to RapidPro

In [650]:
# To show plots in the notebook
%matplotlib inline  

import pandas as pd
import numpy as np
import pandas_highcharts.core
from sqlalchemy import create_engine
import psycopg2
import matplotlib.pyplot 

import os

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "IMAM.settings")

from django.conf import settings
from home.management.commands.load_data import assign_state_lga_num, rename_cols, generic_cleaning, merge_in_and_outpatients, add_program_reports_from_supervision

import django
django.setup()

from home.models import First_admin, Second_admin, Site, Registration

In [651]:
# For exporting excel files using XlsxWriter
from pandas import ExcelWriter
import xlsxwriter

In [652]:
# Suppress scientific notation
# pd.options.display.float_format = '{:20,.0f}'.format

In [983]:
engine = create_engine(
    'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(**settings.DATABASES['default']))
df = pd.read_sql_query("select * from registration;", con=engine)

In [984]:
# If there is no data in PostGres
# df = pd.ExcelFile('/home/robert/Downloads/reg.xlsx').parse('Contacts')

In [985]:
df.head()

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail,lga_num,state_num
0,7903e497-8bfe-47a5-bfd6-78584b3cc09f,tel:+2348165422728,Munirat Makama.,,2003110030,OTP,2016-10-31 10:54:37.155412,2017-04-24 08:10:10.429518,Nurse/Midwife,None,2003.0,20.0
1,069c45bd-3e28-4461-a3a2-da998681f501,tel:+2347064388816,DAYYABU MUSA.,None,3510110005,OTP,2016-12-12 06:40:36.221166,2017-05-07 13:57:44.327906,Community Health Officer,None,3510.0,35.0
2,a591122c-7319-4d2c-85e3-c28d5fdd44a0,tel:+2348034042391,Falmata Ali Mustapha.,,813110003,OTP,2016-10-14 10:38:05.292361,2017-05-05 14:22:03.666872,Community Health Officer,faltalimutapha202@gmail.com,813.0,8.0
3,e978059a-ffdd-44b8-84a7-8ce8c0aed7af,tel:+2348180200915,Abdulkadir Muhammad Yasore,None,2003,Sup,2016-11-05 16:33:04.907624,2017-05-08 12:38:59.153229,None,akyasore@gmail.com,2003.0,20.0
4,3130d94b-f3bd-4412-b7c9-acd8b5598e81,tel:+2348087760645,IBRAHIM LAMARA.,,504110010,OTP,2017-02-22 14:12:33.933315,2017-04-12 14:09:33.640518,In Charge Hospital/PHC,None,504.0,5.0


In [986]:
# run rename columns function
rename_cols(df)

,contact_uuid,urn,name,groups,siteid,type,first_seen,last_seen,post,mail,lga_num,state_num
0,7903e497-8bfe-47a5-bfd6-78584b3cc09f,tel:+2348165422728,Munirat Makama.,,2003110030,OTP,2016-10-31 10:54:37.155412,2017-04-24 08:10:10.429518,Nurse/Midwife,None,2003.0,20.0
1,069c45bd-3e28-4461-a3a2-da998681f501,tel:+2347064388816,DAYYABU MUSA.,None,3510110005,OTP,2016-12-12 06:40:36.221166,2017-05-07 13:57:44.327906,Community Health Officer,None,3510.0,35.0
2,a591122c-7319-4d2c-85e3-c28d5fdd44a0,tel:+2348034042391,Falmata Ali Mustapha.,,813110003,OTP,2016-10-14 10:38:05.292361,2017-05-05 14:22:03.666872,Community Health Officer,faltalimutapha202@gmail.com,813.0,8.0
3,e978059a-ffdd-44b8-84a7-8ce8c0aed7af,tel:+2348180200915,Abdulkadir Muhammad Yasore,None,2003,Sup,2016-11-05 16:33:04.907624,2017-05-08 12:38:59.153229,None,akyasore@gmail.com,2003.0,20.0
4,3130d94b-f3bd-4412-b7c9-acd8b5598e81,tel:+2348087760645,IBRAHIM LAMARA.,,504110010,OTP,2017-02-22 14:12:33.933315,2017-04-12 14:09:33.640518,In Charge Hospital/PHC,None,504.0,5.0
5,227ec725-6868-4ee3-9589-a2860c72daa8,tel:+2349027660360,EKENTA FANNY NJIDEKA.,,807210007,SC,2017-04-28 10:03:59.903480,2017-04-28 14:41:27.358756,Nurse/Midwife,nekenta@internationalmedicalcorps.org,807.0,8.0
6,a02518c4-935b-46cf-a1ce-11d23ba6d38b,tel:+2347033361497,JOY B MBAYA.,,821210017,SC,2017-04-28 10:26:51.567596,2017-04-28 14:37:43.264403,Community Health Officer,None,821.0,8.0
7,b0670cfa-c8ef-4e1d-8eb2-d3ab70efe2eb,tel:+2348086596462,Mannir Ibrahim.,,3605110063,OTP,2016-11-07 08:41:14.014344,2017-01-18 20:35:52.997439,Community Health Officer,None,3605.0,36.0
8,dfae6793-c610-4f33-bd27-952f259c92b1,tel:+2348069613783,Sani Magaji Nahuce.,,3605110056,OTP,2016-11-08 08:13:54.081639,2017-01-18 20:35:46.335767,None,None,3605.0,36.0
9,2a82f74b-c2ef-465b-9f8f-f86dabce6ef1,tel:+2348080796839,Musa Abdullahi.,,3603110029,OTP,2016-11-06 10:03:24.573761,2017-01-18 20:35:31.345573,None,None,3603.0,36.0


In [987]:
# CHANGE URN TO NUM
df = df.rename(index=str, columns={"urn": "num"})

In [988]:
#df.state_num= df.state_num.astype(int)
# df.lga_num= df.lga_num.astype(int)
# Cannot convert NA to integer


In [989]:
# Check for missing siteids
df.query('siteid!=siteid')

,contact_uuid,num,name,groups,siteid,type,first_seen,last_seen,post,mail,lga_num,state_num


In [990]:
# test if there are any entries with no lga_num (not state level)

dftest = df.query('siteid > 36')
dftest.query('lga_num!=lga_num')

,contact_uuid,num,name,groups,siteid,type,first_seen,last_seen,post,mail,lga_num,state_num
135,c0779ecc-2d44-47db-aefd-87a9f28cbed9,tel:+2348098947772,30.,,99,None,2016-10-17 10:43:47.932219,2017-04-27 14:42:26.561978,None,None,NaN,99.0
485,a5daeb01-7b49-4c0e-b0a1-4f6cc7ee3734,tel:+2348133982823,Abubakar Mustapha.,,99,OTP,2016-09-08 15:28:13.943909,2017-04-03 06:20:22.373171,In Charge Hospital/PHC,None,NaN,99.0
730,0beec717-475d-4247-b2d0-152b47987237,tel:+2348034322013,Shuaibu Aliyu Ringim .,,99,None,2016-09-05 11:08:30.356151,2017-05-02 10:12:42.846297,Observer,shuaibringim@gmail.com,NaN,99.0
756,19ca7858-edcf-4d86-b2af-d1854d5dc90a,tel:+2348037850351,Florence Ebun Oni.,,99,None,2016-12-06 11:33:26.029369,2017-05-02 17:43:41.272834,Technical Assistance,foni@unicef.org,NaN,99.0
908,938527d7-6e4c-4b93-a82a-1621d8eb4771,tel:+2348160845076,Bashar Sa.du.,,99,None,2016-11-24 13:59:06.986443,2017-04-11 08:35:08.386114,None,None,NaN,99.0
910,1c3d177f-52a9-45e4-b00a-21ab9a267d64,tel:+2348084755499,Haliru Isah.,,99,None,2016-07-17 09:05:28.061832,2017-04-11 08:32:30.178801,None,None,NaN,99.0
1154,2e3ab6f7-4bff-411d-9565-fc174a57a7de,tel:+2348065992888,Baba gaji lawan.,,351707,None,2017-01-14 14:35:48.137461,2017-02-13 14:32:11.069752,None,None,NaN,NaN
1656,01e29ad1-87db-471d-9c47-f32691096963,tel:+2348132928554,Omolola Adebayo.,,99,None,2016-11-24 11:11:28.298293,2017-04-28 08:09:24.155527,Observer,None,NaN,99.0
1691,980369db-7e03-41d0-8f73-3909fda60e6e,tel:+2347088790682,Jamil Mohammad.,,43100110,None,2016-11-03 21:28:34.716529,2017-01-16 09:01:50.017487,None,None,NaN,NaN
1692,1112aee2-471a-4a20-8907-0bd25299e515,tel:+2347080867122,Lawan Hassan .,,17002312,None,2016-09-13 15:55:11.776705,2017-01-16 09:01:49.528369,None,None,NaN,NaN


In [991]:
# rename SiteID to siteid
# df=df.rename(columns = {'SiteID':'siteid'})

# Change the order (the index) of the columns
columnsTitles = ['siteid',                 
                 'name',
                 'num',
                 'mail',
                 'post',
                 'type',
                 'first_seen',
                 'last_seen',
                 'lga_num', 
                 'state_num' 
                 ]

df2 = df.reindex(columns=columnsTitles)
df2

,siteid,name,num,mail,post,type,first_seen,last_seen,lga_num,state_num
0,2003110030,Munirat Makama.,tel:+2348165422728,None,Nurse/Midwife,OTP,2016-10-31 10:54:37.155412,2017-04-24 08:10:10.429518,2003.0,20.0
1,3510110005,DAYYABU MUSA.,tel:+2347064388816,None,Community Health Officer,OTP,2016-12-12 06:40:36.221166,2017-05-07 13:57:44.327906,3510.0,35.0
2,813110003,Falmata Ali Mustapha.,tel:+2348034042391,faltalimutapha202@gmail.com,Community Health Officer,OTP,2016-10-14 10:38:05.292361,2017-05-05 14:22:03.666872,813.0,8.0
3,2003,Abdulkadir Muhammad Yasore,tel:+2348180200915,akyasore@gmail.com,None,Sup,2016-11-05 16:33:04.907624,2017-05-08 12:38:59.153229,2003.0,20.0
4,504110010,IBRAHIM LAMARA.,tel:+2348087760645,None,In Charge Hospital/PHC,OTP,2017-02-22 14:12:33.933315,2017-04-12 14:09:33.640518,504.0,5.0
5,807210007,EKENTA FANNY NJIDEKA.,tel:+2349027660360,nekenta@internationalmedicalcorps.org,Nurse/Midwife,SC,2017-04-28 10:03:59.903480,2017-04-28 14:41:27.358756,807.0,8.0
6,821210017,JOY B MBAYA.,tel:+2347033361497,None,Community Health Officer,SC,2017-04-28 10:26:51.567596,2017-04-28 14:37:43.264403,821.0,8.0
7,3605110063,Mannir Ibrahim.,tel:+2348086596462,None,Community Health Officer,OTP,2016-11-07 08:41:14.014344,2017-01-18 20:35:52.997439,3605.0,36.0
8,3605110056,Sani Magaji Nahuce.,tel:+2348069613783,None,None,OTP,2016-11-08 08:13:54.081639,2017-01-18 20:35:46.335767,3605.0,36.0
9,3603110029,Musa Abdullahi.,tel:+2348080796839,None,None,OTP,2016-11-06 10:03:24.573761,2017-01-18 20:35:31.345573,3603.0,36.0


In [992]:
df2.post.value_counts()

# Correct in Registration
# Community Health Worker        1
# hw                             1
# HC                             1

Community Health Officer    1165
In Charge Hospital/PHC       891
Coordinator                  204
Technical Assistance          97
Nurse/Midwife                 75
Volunteer                     64
LabTech-Pharm                 60
Stocks Manager                35
Database Manager              29
Doctor                        17
Observer                      14
STOCKS MANAGER                 2
DATABASE MANAGER               2
In Charge Hospital/Phc         1
Name: post, dtype: int64

In [993]:
# Post ranking
df2['post_rank'] = df2.post.str.lower()

# Coordinator                    1
# Stocks Manager                 2
# Database Manager               3

# In Charge Hospital/PHC         4
# Doctor                         5
# Nurse/Midwife                  6
# LabTech-Pharm                  7
# Community Health Officer       8
# Volunteer                      9
# Technical Assistance          10
# Observer                      11

# Use replace with dictionary
df2['post_rank'].replace({'coordinator': 1,
                        'stocks manager': 2,
                        'database manager': 3,
                        'in charge hospital/phc': 4,
                        'doctor': 5,
                        'nurse/midwife': 6,
                        'labtech-pharm': 7,
                        'community health officer': 8,
                        'volunteer': 9,
                        'technical assistance': 10,
                        'observer': 11}, 
                         inplace=True)

# df2['post_rank'] = df2.post.str.lower().replace({
#                         'coordinator': 1,
#                         'stocks manager': 2,
#                         'database manager': 3,
#                         'in charge hospital/phc': 4,
#                         'doctor': 5,
#                         'nurse/midwife': 6,
#                         'labtech-pharm': 7,
#                         'community health officer': 8,
#                         'volunteer': 9,
#                         'technical assistance': 10,
#                         'observer': 11}, 
#                          inplace=True)


In [994]:
df2['post_rank'].value_counts()
# Used to sort by rank later

8.0     1165
4.0      892
1.0      204
10.0      97
6.0       75
9.0       64
7.0       60
2.0       37
3.0       31
5.0       17
11.0      14
Name: post_rank, dtype: int64

In [995]:
# df2.type
# change type in supervision cadres to "Sup"
df2['type'] = np.where((df2['siteid']) < 3699, "Sup", df2['type'])

In [996]:
# REMEMBER With queries, the evaluation phrase must be a string in single or double quotes
# df2.query('type == ","')
# removed one row with , as type

In [997]:
df2.type.value_counts()


OTP    2186
Sup     357
SC      177
Name: type, dtype: int64

In [998]:
df2['siteid'].count()

2774

In [999]:
df2['type'].unique()
# At least 54 registrations with None as type

array([u'OTP', 'Sup', u'SC', None], dtype=object)

In [1000]:
# Assign state lga num is not necessary as it is done already in the import_contacts.py
# df2 = assign_state_lga_num(df2)
# corrected error with LGA Num in load_data.py

In [1001]:
df2 = df2.query('siteid>1').query('siteid!=99')
# Remove national level registrations
# siteid == 99 is a number to use for missing or no longer involved with IMAM program
# Remove erroneous registrations

In [1002]:
df2 = df2.sort_values(by=['siteid','post_rank', 'name'])

In [1003]:
del supervision_df

In [1004]:
# create db with only supervision staff
supervision_df = df2[df2['siteid'] <= 3699]
# supervision siteids range from 1 to 3699

In [1005]:
# to pivot data, create counts of each case by siteid
# Remember that siteid == state_num or LGA_num if on supervision level. 
# It is not necessary to run this command below on the state_num or lga_num. 

# copy of a slice from a DataFrame - ERROR
# supervision_df['count'] = supervision_df.groupby('siteid').cumcount() + 1
#supervision_df.loc['count'] = supervision_df.groupby('siteid').cumcount() + 1
# df.loc[:, :] is supposedly equivalent to all rows and columns in dataframe

supervision_df.loc[: , 'count'] = supervision_df.groupby('siteid').cumcount() + 1

In [1006]:
# supervision_df['count'] = supervision_df.groupby('siteid').cumcount() + 1

In [1007]:
# supervision_df = supervision_df.drop('type', axis=1)

In [1008]:
# Supervision long dataframe
supervision_df

# num labelled as num

,siteid,name,num,mail,post,type,first_seen,last_seen,lga_num,state_num,post_rank,count
701,2,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,Coordinator,Sup,2016-10-26 09:04:19.462503,2017-05-08 08:22:44.912015,NaN,2.0,1.0,1
2306,2,Hauwa Zoakah.,tel:+2347035678763,hauwabata@yahoo.com,Coordinator,Sup,2016-06-01 12:17:48.149140,2017-05-08 08:22:44.164408,NaN,2.0,1.0,2
2305,2,Ijagila Mark .,tel:+2348100001300,ijagilamark@gmail.com,Coordinator,Sup,2016-10-24 09:36:56.209869,2017-05-08 08:22:44.541012,NaN,2.0,1.0,3
2307,2,Wullanga Alfred,tel:+2347032853473,wulangaalfred@gmail.com,Coordinator,Sup,2016-10-26 09:06:02.356386,2017-05-08 08:22:43.794723,NaN,2.0,1.0,4
702,2,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,Database Manager,Sup,2016-10-24 10:38:14.041832,2017-05-08 08:22:42.640776,NaN,2.0,3.0,5
2309,2,Reuben Aidaticha.,tel:+2348038213859,aidaticha@gmail.com,Database Manager,Sup,2016-10-24 09:35:36.024432,2017-05-08 08:22:43.006681,NaN,2.0,3.0,6
2310,2,Wullanga Alfred.,tel:+2348029864318,wullangaalfred@gmail.com,Database Manager,Sup,2016-10-24 09:35:46.229729,2017-05-08 08:22:42.272719,NaN,2.0,3.0,7
2308,2,Olawumi Monica Ajayi.,tel:+2348036173901,wumi.ajayi@yahoo.com,Technical Assistance,Sup,2016-10-24 09:38:19.635591,2017-05-08 08:22:43.385150,NaN,2.0,10.0,8
254,5,Charity Evans Nysalamke.,tel:+2348036581237,nyalamkecharityevans@yahoo.com,Coordinator,Sup,2016-09-02 10:41:18.971823,2017-02-27 11:33:29.021348,NaN,5.0,1.0,1
621,5,Hamza Yakubu Sade .,tel:+2348037716795,None,Coordinator,Sup,2016-09-02 10:41:35.895429,2017-02-27 11:33:31.265269,NaN,5.0,1.0,2


In [1011]:
# test if num contains any null values
supervision_df['num'].isnull().value_counts()

False    341
Name: num, dtype: int64

In [1012]:
# The data points to include in IMAM Supervision database for each supervision SiteID
# Name, phone number, siteid, sitename, state(name), lga(name), state(supervision), lga(supervision)

columnsTitles = ['name',
                 'num',
                 'siteid',  
                 'mail',
                 'state_num',
                 'lga_num',
                 'count',
                 ]

supervision_df = supervision_df.reindex(columns=columnsTitles)

In [1013]:
supervision_df

,name,num,siteid,mail,state_num,lga_num,count
701,Hauwa Zoakah .,tel:+2348020938959,2,hauwabata@yahoo.com,2.0,NaN,1
2306,Hauwa Zoakah.,tel:+2347035678763,2,hauwabata@yahoo.com,2.0,NaN,2
2305,Ijagila Mark .,tel:+2348100001300,2,ijagilamark@gmail.com,2.0,NaN,3
2307,Wullanga Alfred,tel:+2347032853473,2,wulangaalfred@gmail.com,2.0,NaN,4
702,Reuben Aidaticha.,tel:+2348088481006,2,aidaticha@gmail.com,2.0,NaN,5
2309,Reuben Aidaticha.,tel:+2348038213859,2,aidaticha@gmail.com,2.0,NaN,6
2310,Wullanga Alfred.,tel:+2348029864318,2,wullangaalfred@gmail.com,2.0,NaN,7
2308,Olawumi Monica Ajayi.,tel:+2348036173901,2,wumi.ajayi@yahoo.com,2.0,NaN,8
254,Charity Evans Nysalamke.,tel:+2348036581237,5,nyalamkecharityevans@yahoo.com,5.0,NaN,1
621,Hamza Yakubu Sade .,tel:+2348037716795,5,None,5.0,NaN,2


In [1014]:
# create db with only State Level supervision staff
first_admin = supervision_df[supervision_df['siteid'] <= 39]



In [1015]:
columnsTitles = ['name',
                 'num',
                 'siteid',  
                 'mail',
                 'count',
                 ]

first_admin = first_admin.reindex(columns=columnsTitles)

In [1016]:
first_admin

,name,num,siteid,mail,count
701,Hauwa Zoakah .,tel:+2348020938959,2,hauwabata@yahoo.com,1
2306,Hauwa Zoakah.,tel:+2347035678763,2,hauwabata@yahoo.com,2
2305,Ijagila Mark .,tel:+2348100001300,2,ijagilamark@gmail.com,3
2307,Wullanga Alfred,tel:+2347032853473,2,wulangaalfred@gmail.com,4
702,Reuben Aidaticha.,tel:+2348088481006,2,aidaticha@gmail.com,5
2309,Reuben Aidaticha.,tel:+2348038213859,2,aidaticha@gmail.com,6
2310,Wullanga Alfred.,tel:+2348029864318,2,wullangaalfred@gmail.com,7
2308,Olawumi Monica Ajayi.,tel:+2348036173901,2,wumi.ajayi@yahoo.com,8
254,Charity Evans Nysalamke.,tel:+2348036581237,5,nyalamkecharityevans@yahoo.com,1
621,Hamza Yakubu Sade .,tel:+2348037716795,5,None,2


In [1017]:
# test if num contains any null values
first_admin['num'].isnull().value_counts()

False    102
Name: num, dtype: int64

In [1018]:
# convert vertical to horizontal database. 
# will None in the cells cause us to send excessive number of warning SMS? 
first_admin_wide= first_admin.pivot(index='siteid', columns='count')

In [1019]:
first_admin_wide

name                             \
count                               1                          2    
siteid                                                              
2                       Hauwa Zoakah .              Hauwa Zoakah.   
5             Charity Evans Nysalamke.        Hamza Yakubu Sade .   
8             Hassana Suleiman Jibrin.     Abdullahi Alhaji Madi.   
16      Olufunmilayo Adepoju-adebambo.         Ronas Amos Amusa .   
17           MUSA  MOHAMMED  HADEJIA .          Saidu Umar Adamu.   
18                       Hauwa Usman .                Jane Gwani.   
19                   Ado Jibrin Sanda.              Ado Mustapha.   
20                Rabia Mohammed Sno .        Hamisu Idris K/bai.   
21          Abdulmalik Muhammad Illo .  Abdulmalik Muhammad Illo.   
33                     Abdallah Ladan.             Buharismalami.   
35                        Laraba Audu.               YABAWA ABBA.   
36                      Aliyu Ibrahim.         Bello Umar Jabaka.   

                                                                 \
count                               3                        4    
siteid                                                            
2                       Ijagila Mark .          Wullanga Alfred   
5               Sama'ila Usman Maikan.             Yakubu Baba.   
8       Amarachi, Clementina Chukwuma.     Aminu Usman Danzomo.   
16                    Suleiman Mamman.            Usman Baraya.   
17                  Olatomiwa Olabisi.   Temidayo Esther Ajala.   
18                      Saratu Aduwak.           Maryam Yusuf .   
19                    Murtala M Inuwa.  Murtala Sani Abdullahi.   
20                   Abdulmumin Lawan.    Ibrahim Maje Sayyadi.   
21              Aliyu Galadima Libata.          Beatrice Kwere.   
33                 Muhammad Ali Hamza.              Nura Muazu.   
35                  Ahmed Bulama Musa.    Auwal Ibrahim Jauro .   
36                   Ayobami Oyedeji .         Ayobami Oyedeji.   

                                                              \
count                           5                         6    
siteid                                                         
2                Reuben Aidaticha.         Reuben Aidaticha.   
5                 Ali Shehu Kobi .    Habu Abdulmalik Dauda.   
8                   Daniel James .             Magret Ayuba.   
16             Ibrahim Inuwa Lano.       Ahmed Audu Saddana.   
17                            None                      None   
18                            None                      None   
19                     Akilu Sani.                Sabo Wada.   
20            Yusufahmadu Gangara.      Zayyana Isyaku Sule.   
21                Shamsu Muhammed.    Abimbola Abosede Sobo.   
33      Hassan Muhammad Galadanci.  Hassanmuhammadgaladanci.   
35            Auwal ibrahim jauro.  Adeleye Grace Bunmilola.   
36                Azeezat O. Sule.             John Tsebam .   

                                                           \
count                        7                         8    
siteid                                                      
2              Wullanga Alfred.     Olawumi Monica Ajayi.   
5         Jackson Ladu Martins.                      None   
8                          None                      None   
16            Selamawit Negash.            Rukayya Lawal.   
17                         None                      None   
18                         None                      None   
19                   Sabo Wada.   Abolarin, Samuel Sesan.   
20        Abdulhadi Abdulkadir.  Ado Ibrahim Abdulrahman.   
21        Abisola Mary Atoyebi.   Aliyu Galadima Libata .   
33             Kamaru Muhammad.              Nura Shehu .   
35      Adrienne Colwell Klein.                   Alisia.   
36        Saifullahi Abdullahi.                      None   

                                                           \
count                      9                          

In [1020]:
first_admin_wide.dtypes

      count
name  1        object
      2        object
      3        object
      4        object
      5        object
      6        object
      7        object
      8        object
      9        object
      10       object
      11       object
      12       object
      13       object
      14       object
      15       object
      16       object
      17       object
      18       object
num   1        object
      2        object
      3        object
      4        object
      5        object
      6        object
      7        object
      8        object
      9        object
      10       object
      11       object
      12       object
      13       object
      14       object
      15       object
      16       object
      17       object
      18       object
mail  1        object
      2        object
      3        object
      4        object
      5        object
      6        object
      7        object
      8        object
      9        objec

In [1021]:
# Create new column name
# and correct multiIndex
first_admin_wide.columns = ["sno" + str(first_admin_wide.columns[i][1]) + (first_admin_wide.columns[i][0]) for i in range(len(first_admin_wide.columns))]


In [1022]:
first_admin_wide = first_admin_wide.reset_index()

# Note state_num does not exist, but is same as siteid

In [1023]:
# Prepared state level list of supervisors

In [1024]:
# set index to be state_num - same as siteid for state
first_admin_wide = first_admin_wide.set_index('siteid')
# remove name of index
first_admin_wide.index.name = None


In [1025]:
first_admin_wide

,sno1name,sno2name,sno3name,sno4name,sno5name,sno6name,sno7name,sno8name,sno9name,sno10name,sno11name,sno12name,sno13name,sno14name,sno15name,sno16name,sno17name,sno18name,sno1num,sno2num,sno3num,sno4num,sno5num,sno6num,sno7num,sno8num,sno9num,sno10num,sno11num,sno12num,sno13num,sno14num,sno15num,sno16num,sno17num,sno18num,sno1mail,sno2mail,sno3mail,sno4mail,sno5mail,sno6mail,sno7mail,sno8mail,sno9mail,sno10mail,sno11mail,sno12mail,sno13mail,sno14mail,sno15mail,sno16mail,sno17mail,sno18mail
2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
5,Charity Evans Nysalamke.,Hamza Yakubu Sade .,Sama'ila Usman Maikan.,Yakubu Baba.,Ali Shehu Kobi .,Habu Abdulmalik Dauda.,Jackson Ladu Martins.,None,None,None,None,None,None,None,None,None,None,None,tel:+2348036581237,tel:+2348037716795,tel:+2348038233464,tel:+2348083260530,tel:+2347032683737,tel:+2348180448144,tel:+2348035350951,None,None,None,None,None,None,None,None,None,None,None,nyalamkecharityevans@yahoo.com,None,None,yakubuhardawa14@yahoo.com,None,dhadboolagajei72@gmail.com,jmartins@unicef.org,None,None,None,None,None,None,None,None,None,None,None
8,Hassana Suleiman Jibrin.,Abdullahi Alhaji Madi.,"Amarachi, Clementina Chukwuma.",Aminu Usman Danzomo.,Daniel James .,Magret Ayuba.,None,None,None,None,None,None,None,None,None,None,None,None,tel:+2348065476174,tel:+2349090154070,tel:+2348036509268,tel:+2348064275868,tel:+2347035337582,tel:+2348130221420,None,None,None,None,None,None,None,None,None,None,None,None,None,abdullahimadi@gmail.com,camarachi@ccfng.org,ausman@unicef.org,danieljames858@gmail.com,magreta001@gmail.com,None,None,None,None,None,None,None,None,None,None,None,None
16,Olufunmilayo Adepoju-adebambo.,Ronas Amos Amusa .,Suleiman Mamman.,Usman Baraya.,Ibrahim Inuwa Lano.,Ahmed Audu Saddana.,Selamawit Negash.,Rukayya Lawal.,None,None,None,None,None,None,None,None,None,None,tel:+2348033461351,tel:+2348023803791,tel:+2348039323034,tel:+2348035599416,tel:+2348080440504,tel:+2347062471818,tel:+2348035319233,tel:+2348035138367,None,None,None,None,None,None,None,None,None,None,funmi.tumi.adebambo@gmail.com,adoronge@yahoo.com,smamson@yahoo.com,usbaraya44@gmail.com,None,ahmedaudu2005@yahoo.com,snegash@unicef.org,None,None,None,None,None,None,None,None,None,None,None
17,MUSA MOHAMMED HADEJIA .,Saidu Umar Adamu.,Olatomiwa Olabisi.,Temidayo Esther Ajala.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,tel:+2348065921417,tel:+2348039275863,tel:+2348039155506,tel:+2348036233395,None,None,None,None,None,None,None,None,None,None,None,None,None,None,mohammedmusahja@gmail.com,saiduumaradamu@ymail.com,olatomiwa_olabisi@yahoo.com,bidunesther@yahoo.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None
18,Hauwa Usman .,Jane Gwani.,Saratu Aduwak.,Maryam Yusuf .,None,None,None,None,None,None,None,None,None,None,None,None,None,None,tel:+2348025630812,tel:+2348028332127,tel:+2349077197415,tel:+2348033418102,None,None,None,None,None,None,None,None,None,None,None,None,None,None,hauwausmanu@gmail.com,janegwani@yahoo.com,None,maryamwushishi79@gmail.com,None,None,None,None,None,None,None,None,None,None,None,None,None,None
19,Ado Jibrin Sanda.,Ado Mustapha.,Murtala M Inuwa.,Murtala Sani Abdullahi.,Akilu Sani.,Sabo Wada.,Sabo Wada.,"Abolarin, Samuel Sesan.",Deborah Abi Nyako.,Dorcas Heinmen Gauji.,Fumen Fuset Emmanuel.,Nyako William Azi.,Olalekan Wahab Runmonkun .,Abigail Ishaya Nyam .,Akilu Sani.,Ayodeji Osunkentan.

In [1026]:
# Test for correct phone numbers
first_admin_wide.sno1num.value_counts()

tel:+2348020938959    1
tel:+2348065476174    1
tel:+2348035890806    1
tel:+2348065921417    1
tel:+2348106438203    1
tel:+2347088113257    1
tel:+2348022303369    1
tel:+2347039678456    1
tel:+2348025630812    1
tel:+2348060565916    1
tel:+2348033461351    1
tel:+2348036581237    1
Name: sno1num, dtype: int64

In [1027]:
# Add first_admin_wide to first_admin to merge to IMAM supervision df later
first_admin  = first_admin.drop('count', axis=1)

In [1028]:
# Use column space for sitename
first_admin.rename(columns={'mail': 'sitename'}, inplace=True)

In [1029]:
# Add site name from postgres
first_admin['sitename'] = first_admin['siteid'].map(lambda x: First_admin.objects.get(state_num=x).state.strip() if First_admin.objects.filter(state_num=x) else "")

In [1030]:
first_admin = pd.merge(first_admin,  , left_on='siteid', right_index=True, how='left', sort=False)

In [827]:
first_admin

,name,num,siteid,sitename,sno1name,sno2name,sno3name,sno4name,sno5name,sno6name,...,sno9mail,sno10mail,sno11mail,sno12mail,sno13mail,sno14mail,sno15mail,sno16mail,sno17mail,sno18mail
701,Hauwa Zoakah .,tel:+2348020938959,2,ADAMAWA,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,...,None,None,None,None,None,None,None,None,None,None
2306,Hauwa Zoakah.,tel:+2347035678763,2,ADAMAWA,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,...,None,None,None,None,None,None,None,None,None,None
2305,Ijagila Mark .,tel:+2348100001300,2,ADAMAWA,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,...,None,None,None,None,None,None,None,None,None,None
2307,Wullanga Alfred,tel:+2347032853473,2,ADAMAWA,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,...,None,None,None,None,None,None,None,None,None,None
702,Reuben Aidaticha.,tel:+2348088481006,2,ADAMAWA,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,...,None,None,None,None,None,None,None,None,None,None
2309,Reuben Aidaticha.,tel:+2348038213859,2,ADAMAWA,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,...,None,None,None,None,None,None,None,None,None,None
2310,Wullanga Alfred.,tel:+2348029864318,2,ADAMAWA,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,...,None,None,None,None,None,None,None,None,None,None
2308,Olawumi Monica Ajayi.,tel:+2348036173901,2,ADAMAWA,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,...,None,None,None,None,None,None,None,None,None,None
254,Charity Evans Nysalamke.,tel:+2348036581237,5,BAUCHI,Charity Evans Nysalamke.,Hamza Yakubu Sade .,Sama'ila Usman Maikan.,Yakubu Baba.,Ali Shehu Kobi .,Habu Abdulmalik Dauda.,...,None,None,None,None,None,None,None,None,None,None
621,Hamza Yakubu Sade .,tel:+2348037716795,5,BAUCHI,Charity Evans Nysalamke.,Hamza Yakubu Sade .,Sama'ila Usman Maikan.,Yakubu Baba.,Ali Shehu Kobi .,Habu Abdulmalik Dauda.,...,None,None,None,None,None,None,None,None,None,None


In [1032]:
# test if num contains any null values
first_admin['sno1num'].isnull().value_counts()

False    102
Name: sno1num, dtype: int64

In [1034]:
# Create same list for LGA
# create db with only LGA Level supervision staff
second_admin = supervision_df.query('siteid>=101').query('siteid<=3699')
#lga_df = lga_df[lga_df['siteid'] <= 3699]


In [1035]:
columnsTitles = ['name',
                 'num',
                 'siteid',  
                 'mail',
                 'count',
                 ]

second_admin = second_admin.reindex(columns=columnsTitles)

In [1036]:
second_admin

,name,num,siteid,mail,count
1450,Edina Richard.,tel:+2348131816143,201,None,1
2285,Monday Leasado.,tel:+2348038056048,202,None,1
2286,Monday Leasado.,tel:+2348086625852,202,None,2
2287,Monday Leasado.,tel:+2348065356507,202,None,3
2281,Naaticha Waziri .,tel:+2348037675670,203,naatiwaziri69@gmail.com,1
1405,Rose Zidon .,tel:+2347065739356,204,None,1
1440,Rose Mbamuno Zidon,tel:+2349086559594,204,None,2
2279,Esther Danjuma.,tel:+2347030635580,205,None,1
2280,Esther Danjuma.,tel:+2348025568649,205,None,2
2278,Hannatu B Usman.,tel:+2348064811859,206,hannatubbu@gmail.com,1


In [1037]:
# test if num contains any null values
second_admin['num'].isnull().value_counts()

False    239
Name: num, dtype: int64

In [1038]:
# convert vertical to horizontal database. 
# will None in the cells cause us to send excessive number of warning SMS? 
second_admin_wide = second_admin.pivot(index='siteid', columns='count')

In [1039]:
second_admin_wide

name                         \
count                             1                      2   
siteid                                                       
201                  Edina Richard.                   None   
202                 Monday Leasado.        Monday Leasado.   
203               Naaticha Waziri .                   None   
204                    Rose Zidon .     Rose Mbamuno Zidon   
205                 Esther Danjuma.        Esther Danjuma.   
206                Hannatu B Usman.                   None   
207                Aishatu Samaila.       Aishatu Samaila.   
208               Wubbewasu David .       Apollos Elkanah.   
209         Patience Zomti Douglas.                   None   
210                 Adama Abubakar.     Ularamai Raphael .   
211                Yakubu Suleiman.       Yakubu Suleiman.   
212                Aishatu Yuguda .        Wilson Hosanna.   
213            Barnabas Paul Mbiya.                   None   
214                  Hadiza Gidado.         Hadiza Gidado.   
215               Grace John 72221.                   None   
216                  Pwaveno Eliel.                   None   
217               Hannatu I Koliro.                   None   
218              Blessing Ibrahim .      Blessing Ibrahim.   
219                   Shatu Daniel.                   None   
220            Nacha Bitrus Maksha.                   None   
221               Yuguda Abdullahi.                   None   
504              Ahmed Idi Dagauda.                   None   
512               Ibrahim Nalafiya.                   None   
513              Abdullahi M Bello.                   None   
802                 Rebecca Thliza.        Rebecca Thliza.   
803                 Abba Kawu Goni.                   None   
804                    Hauwa Aliyu.                   None   
805               Aishatu Abubakar.      Aishatu Abubakar.   
806                    Pana Tapchi.                     4.   
807                  Aisha Allamin.         Aisha Allamin.   
...                             ...                    ...   
3306           AMINU NAMAKKA BALLE.   Aminu Namakka Balle.   
3308              Bashiru Abubakar.      Bashiru Abubakar.   
3313                   Amina Bello.      Muhammad Ibrahim.   
3317                  Asabe Shehu .          Asabe Shehu .   
3319             Mukhtar Abdullahi.                   None   
3321                  Rukayya Musa.                   None   
3501     Fatsuma Mustapha Mahammad.                   None   
3502                   Zanna Kasim.                   None   
3503            Umar Umar Muhammad.          Aisha Ahmed .   
3504                   Aisha a ali.                   None   
3505                   Asabe Dauda.                   None   
3506                  Naima Usman .                   None   
3507                    Kulu Bello.           Gambo Barde.   
3508                  Amina Yakubu.        Ali M Abubakar.   
3509                ADAMA ALHASSAN.                   None   
3510             Hadiza Maina Adam.     Hadiza Maina Adam.   
3511           Ngabarama kachallah.                   None   
3512                  Sadiya Dahun.        Abubakar Lampo.   
3513                 Hadiza idriss.         Hadiza idriss.   
3514           HAUWA IBRAHIM CHANA.                   None   
3515                     Rhoda.I.g.       Emmanuel Samuel.   
3516    Abdullahi Mohammed Ibrahim.    IBRAHIM MAI KASUWA.   
3517          Bukar Sadiq Mohammed.  Bukar Sadiq mohammed.   
3602                 Hassan Hakimi.           Usman Bello.   
3603           Abdullahi B Suleman.                   None   
3605                 Haliru Hassan.            Rabiu Umar.   
3607                   Umar Ismail.                   None   
3609             Badamasi Muhammad.                   None   
3611                Hassan Ibrahim.            Nura Aminu.   
3613             Hanisatu Mohammed.                   None   

                                                              

In [1040]:
# Rename columns and correct multiIndex
second_admin_wide.columns = ["lga" + str(second_admin_wide.columns[i][1]) + (second_admin_wide.columns[i][0]) for i in range(len(second_admin_wide.columns))]

In [1041]:
second_admin_wide = second_admin_wide.reset_index()

In [1042]:
second_admin_wide

,siteid,lga1name,lga2name,lga3name,lga4name,lga5name,lga1num,lga2num,lga3num,lga4num,lga5num,lga1mail,lga2mail,lga3mail,lga4mail,lga5mail
0,201,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None
1,202,Monday Leasado.,Monday Leasado.,Monday Leasado.,None,None,tel:+2348038056048,tel:+2348086625852,tel:+2348065356507,None,None,None,None,None,None,None
2,203,Naaticha Waziri .,None,None,None,None,tel:+2348037675670,None,None,None,None,naatiwaziri69@gmail.com,None,None,None,None
3,204,Rose Zidon .,Rose Mbamuno Zidon,None,None,None,tel:+2347065739356,tel:+2349086559594,None,None,None,None,None,None,None,None
4,205,Esther Danjuma.,Esther Danjuma.,None,None,None,tel:+2347030635580,tel:+2348025568649,None,None,None,None,None,None,None,None
5,206,Hannatu B Usman.,None,None,None,None,tel:+2348064811859,None,None,None,None,hannatubbu@gmail.com,None,None,None,None
6,207,Aishatu Samaila.,Aishatu Samaila.,None,None,None,tel:+2348022645708,tel:+2348034154949,None,None,None,None,None,None,None,None
7,208,Wubbewasu David .,Apollos Elkanah.,None,None,None,tel:+2348174118980,tel:+2348029839844,None,None,None,None,apollospola@gmail,None,None,None
8,209,Patience Zomti Douglas.,None,None,None,None,tel:+2348065754427,None,None,None,None,None,None,None,None,None
9,210,Adama Abubakar.,Ularamai Raphael .,None,None,None,tel:+2349078543500,tel:+2347087576976,None,None,None,None,None,None,None,None


In [1043]:
# test if num contains any null values
second_admin_wide['lga1num'].isnull().value_counts()

False    131
Name: lga1num, dtype: int64

In [1044]:
# Add second_admin_wide to second_admin to merge to IMAM supervision df later
second_admin = second_admin.drop('count', axis=1)

In [1045]:
# Use column space for sitename
second_admin.rename(columns={'mail': 'sitename'}, inplace=True)

In [1046]:
# Add site name from postgres
second_admin['sitename'] = second_admin['siteid'].map(lambda x: Second_admin.objects.get(lga_num=x).lga.strip() + " LGA" if Second_admin.objects.filter(lga_num=x) else "")

In [1047]:
second_admin

,name,num,siteid,sitename
1450,Edina Richard.,tel:+2348131816143,201,DEMSA LGA
2285,Monday Leasado.,tel:+2348038056048,202,FUFORE LGA
2286,Monday Leasado.,tel:+2348086625852,202,FUFORE LGA
2287,Monday Leasado.,tel:+2348065356507,202,FUFORE LGA
2281,Naaticha Waziri .,tel:+2348037675670,203,GANYE LGA
1405,Rose Zidon .,tel:+2347065739356,204,GIREI LGA
1440,Rose Mbamuno Zidon,tel:+2349086559594,204,GIREI LGA
2279,Esther Danjuma.,tel:+2347030635580,205,GOMBI LGA
2280,Esther Danjuma.,tel:+2348025568649,205,GOMBI LGA
2278,Hannatu B Usman.,tel:+2348064811859,206,GUYUK LGA


In [1048]:
# test if num contains any null values
second_admin['num'].isnull().value_counts()

False    239
Name: num, dtype: int64

In [1049]:
# second_admin_wide.dtypes

In [1050]:
# Change None in cells to blank

#for cell in lga:
#   lga[cell].fillna(value='', inplace=True)

# lga = lga.fillna
# changes df format and leaves None in place. 
# check documentation

In [1051]:
# Add state_num to LGA df
second_admin_wide['state_num'] = 0

In [1059]:
second_admin_wide.dtypes

siteid        int64
lga1name     object
lga2name     object
lga3name     object
lga4name     object
lga5name     object
lga1num      object
lga2num      object
lga3num      object
lga4num      object
lga5num      object
lga1mail     object
lga2mail     object
lga3mail     object
lga4mail     object
lga5mail     object
state_num     int64
dtype: object

In [1062]:
# Recode state_num to LGA df
second_admin_wide['state_num'] = np.where(second_admin_wide['siteid'] <3699, second_admin_wide['siteid'].astype(str).str[:2], second_admin_wide['state_num'])
second_admin_wide['state_num'] = np.where(second_admin_wide['siteid'] <999, second_admin_wide['siteid'].astype(str).str[:1], second_admin_wide['state_num'])


# Do not forget to cast state_num to INT or merge will not work
second_admin_wide['state_num'] = second_admin_wide['state_num'].astype(int)
second_admin_wide['state_num'].value_counts()

8     21
2     21
35    17
21    15
20    15
17    12
33     9
36     7
19     6
16     3
5      3
18     2
Name: state_num, dtype: int64

In [1063]:
second_admin_wide.groupby('state_num')['siteid'].unique()

state_num
2     [201, 202, 203, 204, 205, 206, 207, 208, 209, ...
5                                       [504, 512, 513]
8     [802, 803, 804, 805, 806, 807, 808, 809, 811, ...
16                                   [1604, 1606, 1609]
17    [1702, 1703, 1704, 1710, 1712, 1714, 1716, 171...
18                                         [1818, 1823]
19                 [1905, 1921, 1928, 1936, 1940, 1944]
20    [2001, 2003, 2004, 2005, 2007, 2010, 2011, 201...
21    [2102, 2103, 2104, 2105, 2106, 2108, 2109, 211...
33    [3301, 3304, 3305, 3306, 3308, 3313, 3317, 331...
35    [3501, 3502, 3503, 3504, 3505, 3506, 3507, 350...
36           [3602, 3603, 3605, 3607, 3609, 3611, 3613]
Name: siteid, dtype: object

In [1053]:
second_admin_wide.dtypes

# statenum must be int to merge


siteid        int64
lga1name     object
lga2name     object
lga3name     object
lga4name     object
lga5name     object
lga1num      object
lga2num      object
lga3num      object
lga4num      object
lga5num      object
lga1mail     object
lga2mail     object
lga3mail     object
lga4mail     object
lga5mail     object
state_num     int64
dtype: object

In [1054]:
# Prepared LGA level supervision list


In [1064]:
# merge state df to lga df

# imam_sup = pd.merge(lga,state, on='state_num', suffixes=['_lga','_st'])

# imam_sup = pd.merge(lga, state, on='state_num')


# imam_sup = lga.join(state, how='inner')

# imam_sup = lga.merge(state)

# imam_sup = lga.merge(state, on=['state_num'])

# imam_sup = lga.join(state, on=['state_num'])

first_second = pd.merge(second_admin_wide, first_admin_wide, left_on='state_num', right_index=True, how='left', sort=False)

# imam_sup = lga.join(state, on='state_num')

first_second

,siteid,lga1name,lga2name,lga3name,lga4name,lga5name,lga1num,lga2num,lga3num,lga4num,lga5num,lga1mail,lga2mail,lga3mail,lga4mail,lga5mail,state_num,sno1name,sno2name,sno3name,sno4name,sno5name,sno6name,sno7name,sno8name,sno9name,sno10name,sno11name,sno12name,sno13name,sno14name,sno15name,sno16name,sno17name,sno18name,sno1num,sno2num,sno3num,sno4num,sno5num,sno6num,sno7num,sno8num,sno9num,sno10num,sno11num,sno12num,sno13num,sno14num,sno15num,sno16num,sno17num,sno18num,sno1mail,sno2mail,sno3mail,sno4mail,sno5mail,sno6mail,sno7mail,sno8mail,sno9mail,sno10mail,sno11mail,sno12mail,sno13mail,sno14mail,sno15mail,sno16mail,sno17mail,sno18mail
0,201,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
1,202,Monday Leasado.,Monday Leasado.,Monday Leasado.,None,None,tel:+2348038056048,tel:+2348086625852,tel:+2348065356507,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
2,203,Naaticha Waziri .,None,None,None,None,tel:+2348037675670,None,None,None,None,naatiwaziri69@gmail.com,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
3,204,Rose Zidon .,Rose Mbamuno Zidon,None,None,None,tel:+2347065739356,tel:+2349086559594,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
4,205,Esther Danjuma.,Esther Danjuma.,None,None,None,tel:+2347030635580,tel:+2348025568649,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,

In [1065]:
# test if num contains any null values
first_second['sno1num'].isnull().value_counts()

False    131
Name: sno1num, dtype: int64

In [1067]:
first_second.query('sno1num != sno1num')

,siteid,lga1name,lga2name,lga3name,lga4name,lga5name,lga1num,lga2num,lga3num,lga4num,lga5num,lga1mail,lga2mail,lga3mail,lga4mail,lga5mail,state_num,sno1name,sno2name,sno3name,sno4name,sno5name,sno6name,sno7name,sno8name,sno9name,sno10name,sno11name,sno12name,sno13name,sno14name,sno15name,sno16name,sno17name,sno18name,sno1num,sno2num,sno3num,sno4num,sno5num,sno6num,sno7num,sno8num,sno9num,sno10num,sno11num,sno12num,sno13num,sno14num,sno15num,sno16num,sno17num,sno18num,sno1mail,sno2mail,sno3mail,sno4mail,sno5mail,sno6mail,sno7mail,sno8mail,sno9mail,sno10mail,sno11mail,sno12mail,sno13mail,sno14mail,sno15mail,sno16mail,sno17mail,sno18mail


In [1113]:
# test if num contains any null values
assert True not in first_second['lga1num'].isnull().value_counts().keys()

In [1069]:
# is Statename# null ? 
first_second.sno3name.value_counts()

Amarachi, Clementina Chukwuma.    21
Ijagila Mark .                    21
Ahmed Bulama Musa.                17
Abdulmumin Lawan.                 15
Aliyu Galadima Libata.            15
Olatomiwa Olabisi.                12
Muhammad Ali Hamza.                9
Ayobami Oyedeji .                  7
Murtala M Inuwa.                   6
Suleiman Mamman.                   3
Sama'ila Usman Maikan.             3
Saratu Aduwak.                     2
Name: sno3name, dtype: int64

In [1070]:
# # Test Export as Excel 
# filename = "IMAM_state_lga_test.xlsx"
# writer = pd.ExcelWriter(filename, engine='xlsxwriter')
# imam_sup.to_excel(writer,'Sheet1')
# writer.save()
# writer.close()

In [1071]:
# SiteID is now NaN for state. 
# can convert this to SiteID later.
first_second['lga_num'] = first_second['siteid'] 

# imam_sup.state_num.value_counts()

In [1072]:
first_second['lga_num'].unique()

array([ 201,  202,  203,  204,  205,  206,  207,  208,  209,  210,  211,
        212,  213,  214,  215,  216,  217,  218,  219,  220,  221,  504,
        512,  513,  802,  803,  804,  805,  806,  807,  808,  809,  811,
        812,  813,  814,  815,  816,  818,  819,  820,  821,  824,  825,
        827, 1604, 1606, 1609, 1702, 1703, 1704, 1710, 1712, 1714, 1716,
       1717, 1719, 1720, 1724, 1727, 1818, 1823, 1905, 1921, 1928, 1936,
       1940, 1944, 2001, 2003, 2004, 2005, 2007, 2010, 2011, 2015, 2016,
       2018, 2024, 2026, 2027, 2033, 2034, 2102, 2103, 2104, 2105, 2106,
       2108, 2109, 2111, 2112, 2113, 2114, 2115, 2117, 2118, 2119, 3301,
       3304, 3305, 3306, 3308, 3313, 3317, 3319, 3321, 3501, 3502, 3503,
       3504, 3505, 3506, 3507, 3508, 3509, 3510, 3511, 3512, 3513, 3514,
       3515, 3516, 3517, 3602, 3603, 3605, 3607, 3609, 3611, 3613])

In [1073]:
# Merge first_second to second_admin to append to IMAM Supervision
second_admin = pd.merge(second_admin, first_second, left_on='siteid', right_on='siteid', how='left', sort=False)
# Check suffixes


In [1074]:
second_admin

,name,num,siteid,sitename,lga1name,lga2name,lga3name,lga4name,lga5name,lga1num,lga2num,lga3num,lga4num,lga5num,lga1mail,lga2mail,lga3mail,lga4mail,lga5mail,state_num,sno1name,sno2name,sno3name,sno4name,sno5name,sno6name,sno7name,sno8name,sno9name,sno10name,sno11name,sno12name,sno13name,sno14name,sno15name,sno16name,sno17name,sno18name,sno1num,sno2num,sno3num,sno4num,sno5num,sno6num,sno7num,sno8num,sno9num,sno10num,sno11num,sno12num,sno13num,sno14num,sno15num,sno16num,sno17num,sno18num,sno1mail,sno2mail,sno3mail,sno4mail,sno5mail,sno6mail,sno7mail,sno8mail,sno9mail,sno10mail,sno11mail,sno12mail,sno13mail,sno14mail,sno15mail,sno16mail,sno17mail,sno18mail,lga_num
0,Edina Richard.,tel:+2348131816143,201,DEMSA LGA,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None,201
1,Monday Leasado.,tel:+2348038056048,202,FUFORE LGA,Monday Leasado.,Monday Leasado.,Monday Leasado.,None,None,tel:+2348038056048,tel:+2348086625852,tel:+2348065356507,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None,202
2,Monday Leasado.,tel:+2348086625852,202,FUFORE LGA,Monday Leasado.,Monday Leasado.,Monday Leasado.,None,None,tel:+2348038056048,tel:+2348086625852,tel:+2348065356507,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None,202
3,Monday Leasado.,tel:+2348065356507,202,FUFORE LGA,Monday Leasado.,Monday Leasado.,Monday Leasado.,None,None,tel:+2348038056048,tel:+2348086625852,tel:+2348065356507,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None,202
4,Naaticha Waziri .,tel:+2348037675670,203,GANYE LGA,Naaticha Waziri .,None,None,None,None,tel:+2348037675670,None,N

In [1075]:
# merge to implementation staff df
site = df2.query('siteid > 3699')

In [1078]:
site.lga_num.unique()
# Problem with lga_num as NaN

array([  101.,   201.,   202.,   203.,   204.,   205.,   206.,   207.,
         208.,   209.,   210.,   211.,   212.,   213.,   214.,   215.,
         216.,   217.,   218.,   219.,   220.,   221.,   504.,   512.,
         513.,   802.,   803.,   804.,   805.,   806.,   807.,   808.,
         809.,   811.,   812.,   813.,   814.,   815.,   816.,   818.,
         819.,   820.,   821.,   823.,   824.,   825.,   827.,   837.,
        1604.,  1606.,  1609.,  1702.,  1703.,  1704.,  1706.,  1709.,
        1710.,  1712.,  1713.,  1714.,  1716.,  1717.,  1719.,  1720.,
        1724.,  1727.,  1731.,  1801.,  1802.,  1805.,  1807.,  1808.,
        1809.,  1818.,  1823.,  1905.,  1921.,  1928.,  1929.,  1930.,
        1936.,  1940.,  1944.,  2001.,  2003.,  2004.,  2005.,  2007.,
        2010.,  2011.,  2015.,  2016.,  2018.,  2021.,  2024.,  2026.,
        2027.,  2033.,  2034.,  2102.,  2103.,  2104.,  2105.,  2106.,
        2108.,  2109.,  2111.,  2112.,  2113.,  2114.,  2115.,  2116.,
      

In [1077]:
# Drop data with no lga_num - this deletes entire row where a NaN is found
site = site.query('lga_num==lga_num')

In [1079]:
site.dtypes


siteid                 int64
name                  object
num                   object
mail                  object
post                  object
type                  object
first_seen    datetime64[ns]
last_seen     datetime64[ns]
lga_num              float64
state_num            float64
post_rank            float64
dtype: object

In [1080]:
# Change the order (the index) of the columns
columnsTitles = ['name',
                 'num',
                 'siteid', 
                 'sitename',
                 'lga_num', 
                 'state_num' 
                 ]

site = site.reindex(columns=columnsTitles)

In [1081]:
site

,name,num,siteid,sitename,lga_num,state_num
732,Assay T Bulti.,tel:+2348035351744,101110001,NaN,101.0,1.0
1384,Elfriede Mamie Kormawa.,tel:+2347064019648,101110001,NaN,101.0,1.0
1120,Aisha Kaka Bello.,tel:+2348035865826,101110001,NaN,101.0,1.0
2103,Assurance David.,tel:+2349021245510,201110007,NaN,201.0,2.0
677,Gracegidado.,tel:+2348134107585,201110007,NaN,201.0,2.0
1687,Phcc Dong.,tel:+2349028026299,201110007,NaN,201.0,2.0
1175,Jasper Gift Jonathan.,tel:+2348160018409,201110009,NaN,201.0,2.0
1516,Visia Ma Ambidi.,tel:+2348165753101,201110009,NaN,201.0,2.0
2017,Elcy Jemuel.,tel:+2348092914506,201110021,NaN,201.0,2.0
2018,Regina Nathan.,tel:+2349091141475,201110021,NaN,201.0,2.0


In [1082]:
# merge supervision df to implementation df

# imam_supervision = imam_imp.join(imam_sup, suffixes=('_x', '_y'), on='lga_num')

imam_supervision = pd.merge(site, first_second, left_on='lga_num', right_on='lga_num', how='right', 
                            suffixes=('', '_y'), sort=False)


In [1084]:
# Name, Phone, SiteID, SiteName, State, LGA, LGA name phone email, State name phone email

#imam_supervision.rename(columns={'siteid_x': 'siteid', 'state_num_x': 'state_num'}, inplace=True)

imam_supervision.drop('siteid_y', axis=1, inplace=True)


In [1085]:
imam_supervision = imam_supervision.sort_values(by=['siteid', 'name'])

In [1086]:
imam_supervision

,name,num,siteid,sitename,lga_num,state_num,lga1name,lga2name,lga3name,lga4name,lga5name,lga1num,lga2num,lga3num,lga4num,lga5num,lga1mail,lga2mail,lga3mail,lga4mail,lga5mail,state_num_y,sno1name,sno2name,sno3name,sno4name,sno5name,sno6name,sno7name,sno8name,sno9name,sno10name,sno11name,sno12name,sno13name,sno14name,sno15name,sno16name,sno17name,sno18name,sno1num,sno2num,sno3num,sno4num,sno5num,sno6num,sno7num,sno8num,sno9num,sno10num,sno11num,sno12num,sno13num,sno14num,sno15num,sno16num,sno17num,sno18num,sno1mail,sno2mail,sno3mail,sno4mail,sno5mail,sno6mail,sno7mail,sno8mail,sno9mail,sno10mail,sno11mail,sno12mail,sno13mail,sno14mail,sno15mail,sno16mail,sno17mail,sno18mail
0,Assurance David.,tel:+2349021245510,201110007,NaN,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
1,Gracegidado.,tel:+2348134107585,201110007,NaN,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
2,Phcc Dong.,tel:+2349028026299,201110007,NaN,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
3,Jasper Gift Jonathan.,tel:+2348160018409,201110009,NaN,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
4,Visia Ma Ambidi.,tel:+2348165753101,201110009,NaN,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidati

In [1087]:
# Test for correct merge
a = imam_supervision['sno1num'].isnull()
# urn used instead of num
a

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
10      False
9       False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
23      False
21      False
20      False
22      False
26      False
24      False
25      False
27      False
28      False
29      False
        ...  
2332    False
2331    False
2334    False
2333    False
2335    False
2336    False
2337    False
2338    False
2339    False
2340    False
2341    False
2342    False
2343    False
2344    False
2345    False
2346    False
2348    False
2347    False
2351    False
2349    False
2350    False
2353    False
2352    False
2355    False
2354    False
2357    False
2356    False
2358    False
2359    False
2360    False
Name: sno1num, dtype: bool

In [1088]:
a.value_counts()

False    2361
Name: sno1num, dtype: int64

In [981]:
imam_supervision

,name,num,siteid,sitename,lga_num,state_num,lga1name,lga2name,lga3name,lga4name,lga5name,lga1num,lga2num,lga3num,lga4num,lga5num,lga1mail,lga2mail,lga3mail,lga4mail,lga5mail,state_num_y,sno1name,sno2name,sno3name,sno4name,sno5name,sno6name,sno7name,sno8name,sno9name,sno10name,sno11name,sno12name,sno13name,sno14name,sno15name,sno16name,sno17name,sno18name,sno1num,sno2num,sno3num,sno4num,sno5num,sno6num,sno7num,sno8num,sno9num,sno10num,sno11num,sno12num,sno13num,sno14num,sno15num,sno16num,sno17num,sno18num,sno1mail,sno2mail,sno3mail,sno4mail,sno5mail,sno6mail,sno7mail,sno8mail,sno9mail,sno10mail,sno11mail,sno12mail,sno13mail,sno14mail,sno15mail,sno16mail,sno17mail,sno18mail
0,Assurance David.,tel:+2349021245510,201110007,NaN,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,20,Rabia Mohammed Sno .,Hamisu Idris K/bai.,Abdulmumin Lawan.,Ibrahim Maje Sayyadi.,Yusufahmadu Gangara.,Zayyana Isyaku Sule.,Abdulhadi Abdulkadir.,Ado Ibrahim Abdulrahman.,Musbahu Hamisu.,Sim Karla Sakyenu.,None,None,None,None,None,None,None,None,tel:+2348035890806,tel:+2348034677471,tel:+2348125688424,tel:+2348036378913,tel:+2348038421237,tel:+2348160646783,tel:+2348060821551,tel:+2348188401965,tel:+2348060647427,tel:+2347030663720,None,None,None,None,None,None,None,None,rabiadaura77@yahoo.com,hamisuhara@yahoo.com,abdulmuminlawan@gmail.com,ibrahimmajes@hotmail.com,yusufahmadu5@gmail.com,zayyanais@gmailcom,hadabdul@yahoo.co.uk,ibrado30@gmail.com,hmusbahu@yahoo.com,simkarla@live.com,None,None,None,None,None,None,None,None
1,Gracegidado.,tel:+2348134107585,201110007,NaN,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,20,Rabia Mohammed Sno .,Hamisu Idris K/bai.,Abdulmumin Lawan.,Ibrahim Maje Sayyadi.,Yusufahmadu Gangara.,Zayyana Isyaku Sule.,Abdulhadi Abdulkadir.,Ado Ibrahim Abdulrahman.,Musbahu Hamisu.,Sim Karla Sakyenu.,None,None,None,None,None,None,None,None,tel:+2348035890806,tel:+2348034677471,tel:+2348125688424,tel:+2348036378913,tel:+2348038421237,tel:+2348160646783,tel:+2348060821551,tel:+2348188401965,tel:+2348060647427,tel:+2347030663720,None,None,None,None,None,None,None,None,rabiadaura77@yahoo.com,hamisuhara@yahoo.com,abdulmuminlawan@gmail.com,ibrahimmajes@hotmail.com,yusufahmadu5@gmail.com,zayyanais@gmailcom,hadabdul@yahoo.co.uk,ibrado30@gmail.com,hmusbahu@yahoo.com,simkarla@live.com,None,None,None,None,None,None,None,None
2,Phcc Dong.,tel:+2349028026299,201110007,NaN,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,20,Rabia Mohammed Sno .,Hamisu Idris K/bai.,Abdulmumin Lawan.,Ibrahim Maje Sayyadi.,Yusufahmadu Gangara.,Zayyana Isyaku Sule.,Abdulhadi Abdulkadir.,Ado Ibrahim Abdulrahman.,Musbahu Hamisu.,Sim Karla Sakyenu.,None,None,None,None,None,None,None,None,tel:+2348035890806,tel:+2348034677471,tel:+2348125688424,tel:+2348036378913,tel:+2348038421237,tel:+2348160646783,tel:+2348060821551,tel:+2348188401965,tel:+2348060647427,tel:+2347030663720,None,None,None,None,None,None,None,None,rabiadaura77@yahoo.com,hamisuhara@yahoo.com,abdulmuminlawan@gmail.com,ibrahimmajes@hotmail.com,yusufahmadu5@gmail.com,zayyanais@gmailcom,hadabdul@yahoo.co.uk,ibrado30@gmail.com,hmusbahu@yahoo.com,simkarla@live.com,None,None,None,None,None,None,None,None
3,Jasper Gift Jonathan.,tel:+2348160018409,201110009,NaN,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,20,Rabia Mohammed Sno .,Hamisu Idris K/bai.,Abdulmumin Lawan.,Ibrahim Maje Sayyadi.,Yusufahmadu Gangara.,Zayyana Isyaku Sule.,Abdulhadi Abdulkadir.,Ado Ibrahim Abdulrahman.,Musbahu Hamisu.,Sim Karla Sakyenu.,None,None,None,None,None,None,None,None,tel:+2348035890806,tel:+2348034677471,tel:+2348125688424,tel:+2348036378913,tel:+2348038421237,tel:+2348160646783,tel:+2348060821551,tel:+2348188401965,tel:+2348060647427,tel:+2347030663720,None,None,None,None,None,None,None,None,rabiad

In [1089]:
# only takes NaN into account, NOT None
test = imam_supervision.query('sno1num!=sno1num')
test

,name,num,siteid,sitename,lga_num,state_num,lga1name,lga2name,lga3name,lga4name,lga5name,lga1num,lga2num,lga3num,lga4num,lga5num,lga1mail,lga2mail,lga3mail,lga4mail,lga5mail,state_num_y,sno1name,sno2name,sno3name,sno4name,sno5name,sno6name,sno7name,sno8name,sno9name,sno10name,sno11name,sno12name,sno13name,sno14name,sno15name,sno16name,sno17name,sno18name,sno1num,sno2num,sno3num,sno4num,sno5num,sno6num,sno7num,sno8num,sno9num,sno10num,sno11num,sno12num,sno13num,sno14num,sno15num,sno16num,sno17num,sno18num,sno1mail,sno2mail,sno3mail,sno4mail,sno5mail,sno6mail,sno7mail,sno8mail,sno9mail,sno10mail,sno11mail,sno12mail,sno13mail,sno14mail,sno15mail,sno16mail,sno17mail,sno18mail


In [1090]:
# What state are errors coming from 
test.state_num.value_counts()

Series([], Name: state_num, dtype: int64)

In [979]:
pd.set_option('display.max_columns', 100)

In [978]:
 pd.describe_option('display')

display.chop_threshold : float or None
    if set to a float value, all float values smaller then the given threshold
    will be displayed as exactly 0 by repr and friends.
    [default: None] [currently: None]

display.colheader_justify : 'left'/'right'
    Controls the justification of column headers. used by DataFrameFormatter.
    [default: right] [currently: right]

display.column_space No description available.
    [default: 12] [currently: 12]

display.date_dayfirst : boolean
    When True, prints and parses dates with the day first, eg 20/01/2005
    [default: False] [currently: False]

display.date_yearfirst : boolean
    When True, prints and parses dates with the year first, eg 2005/01/20
    [default: False] [currently: False]

display.encoding : str/unicode
    Defaults to the detected encoding of the console.
    Specifies the encoding to be used for strings returned by to_string,
    these are generally strings meant to be displayed on the console.
    [default: UTF-8] 

In [1091]:
imam_supervision.sno1num.unique()

array([u'tel:+2348020938959', u'tel:+2348036581237', u'tel:+2348065476174',
       u'tel:+2348033461351', u'tel:+2348065921417', u'tel:+2348025630812',
       u'tel:+2348022303369', u'tel:+2348035890806', u'tel:+2347039678456',
       u'tel:+2348060565916', u'tel:+2347088113257', u'tel:+2348106438203'], dtype=object)

In [1092]:
imam_supervision.lga1num.unique()


array([u'tel:+2348131816143', u'tel:+2348038056048', u'tel:+2348037675670',
       u'tel:+2347065739356', u'tel:+2347030635580', u'tel:+2348064811859',
       u'tel:+2348022645708', u'tel:+2348174118980', u'tel:+2348065754427',
       u'tel:+2349078543500', u'tel:+2348035368149', u'tel:+2348037051093',
       u'tel:+2348027705902', u'tel:+2348147144234', u'tel:+2347062001062',
       u'tel:+2347031206232', u'tel:+2348066076411', u'tel:+2347064219553',
       u'tel:+2348165916091', u'tel:+2348060765589', u'tel:+2348134135289',
       u'tel:+2348104983873', u'tel:+2348068249310', u'tel:+2348166434842',
       u'tel:+2348130521995', u'tel:+2349074733517', u'tel:+2349082509065',
       u'tel:+2347039632824', u'tel:+2348124911800', u'tel:+2349023862306',
       u'tel:+2349020877364', u'tel:+2348028764288', u'tel:+2348021130573',
       u'tel:+2348092090642', u'tel:+2348103514343', u'tel:+2348086089550',
       u'tel:+2348068989261', u'tel:+2348029437389', u'tel:+2348091841242',
       u'tel

In [ ]:
# Check for blanks instead of None in cells

# for cell in imam_supervision:
#    state[cell].fillna(value='', inplace=True)

In [ ]:
# Ensure that we have state and LGA level included individually and merged to site level
# Append

In [1095]:
imam_supervision


,name,num,siteid,sitename,lga_num,state_num,lga1name,lga2name,lga3name,lga4name,lga5name,lga1num,lga2num,lga3num,lga4num,lga5num,lga1mail,lga2mail,lga3mail,lga4mail,lga5mail,state_num_y,sno1name,sno2name,sno3name,sno4name,sno5name,sno6name,sno7name,sno8name,sno9name,sno10name,sno11name,sno12name,sno13name,sno14name,sno15name,sno16name,sno17name,sno18name,sno1num,sno2num,sno3num,sno4num,sno5num,sno6num,sno7num,sno8num,sno9num,sno10num,sno11num,sno12num,sno13num,sno14num,sno15num,sno16num,sno17num,sno18num,sno1mail,sno2mail,sno3mail,sno4mail,sno5mail,sno6mail,sno7mail,sno8mail,sno9mail,sno10mail,sno11mail,sno12mail,sno13mail,sno14mail,sno15mail,sno16mail,sno17mail,sno18mail
0,Assurance David.,tel:+2349021245510,201110007,DONG HEALTH CLINIC,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
1,Gracegidado.,tel:+2348134107585,201110007,DONG HEALTH CLINIC,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
2,Phcc Dong.,tel:+2349028026299,201110007,DONG HEALTH CLINIC,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
3,Jasper Gift Jonathan.,tel:+2348160018409,201110009,NEW DEMSA HEALTH CLINIC,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,None,None,2,Hauwa Zoakah .,Hauwa Zoakah.,Ijagila Mark .,Wullanga Alfred,Reuben Aidaticha.,Reuben Aidaticha.,Wullanga Alfred.,Olawumi Monica Ajayi.,None,None,None,None,None,None,None,None,None,None,tel:+2348020938959,tel:+2347035678763,tel:+2348100001300,tel:+2347032853473,tel:+2348088481006,tel:+2348038213859,tel:+2348029864318,tel:+2348036173901,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,hauwabata@yahoo.com,ijagilamark@gmail.com,wulangaalfred@gmail.com,aidaticha@gmail.com,aidaticha@gmail.com,wullangaalfred@gmail.com,wumi.ajayi@yahoo.com,None,None,None,None,None,None,None,None,None,None
4,Visia Ma Ambidi.,tel:+2348165753101,201110009,NEW DEMSA HEALTH CLINIC,201.0,2.0,Edina Richard.,None,None,None,None,tel:+2348131816143,None,None,None,None,None,None,None,

In [791]:
imam_supervision.dtypes


name            object
num             object
siteid           int64
sitename       float64
lga_num        float64
state_num      float64
lga1name        object
lga2name        object
lga3name        object
lga4name        object
lga5name        object
lga1num         object
lga2num         object
lga3num         object
lga4num         object
lga5num         object
lga1mail        object
lga2mail        object
lga3mail        object
lga4mail        object
lga5mail        object
state_num_y      int64
sno1name        object
sno2name        object
sno3name        object
sno4name        object
sno5name        object
sno6name        object
sno7name        object
sno8name        object
                ...   
sno7num         object
sno8num         object
sno9num         object
sno10num        object
sno11num        object
sno12num        object
sno13num        object
sno14num        object
sno15num        object
sno16num        object
sno17num        object
sno18num        object
sno1mail   

In [792]:
# Change the order (the index) of the columns

# columnsTitles = ['name',
#                  'num',
#                  'siteid', 
#                  'sitename',
#                  'lga_num', 
#                  'state_num' 
#                  ]


# site = site.reindex(columns=columnsTitles)

In [1094]:
# add sitenames 
imam_supervision['sitename'] = imam_supervision['siteid'].map(lambda x: Site.objects.get(siteid=x).sitename.strip() if Site.objects.filter(siteid=x) else "")

In [1101]:
# Append first_admin
final_df = second_admin.append(first_admin).append(imam_supervision)

In [1109]:
# Drop state_num lga_num state_num_y
final_df = final_df.drop(['state_num', 'lga_num', 'state_num_y'], axis = 1)


,lga1mail,lga1name,lga1num,lga2mail,lga2name,lga2num,lga3mail,lga3name,lga3num,lga4mail,lga4name,lga4num,lga5mail,lga5name,lga5num,name,num,siteid,sitename,sno10mail,sno10name,sno10num,sno11mail,sno11name,sno11num,sno12mail,sno12name,sno12num,sno13mail,sno13name,sno13num,sno14mail,sno14name,sno14num,sno15mail,sno15name,sno15num,sno16mail,sno16name,sno16num,sno17mail,sno17name,sno17num,sno18mail,sno18name,sno18num,sno1mail,sno1name,sno1num,sno2mail,sno2name,sno2num,sno3mail,sno3name,sno3num,sno4mail,sno4name,sno4num,sno5mail,sno5name,sno5num,sno6mail,sno6name,sno6num,sno7mail,sno7name,sno7num,sno8mail,sno8name,sno8num,sno9mail,sno9name,sno9num
0,None,Edina Richard.,tel:+2348131816143,None,None,None,None,None,None,None,None,None,None,None,None,Edina Richard.,tel:+2348131816143,201,DEMSA LGA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,Hauwa Zoakah.,tel:+2347035678763,ijagilamark@gmail.com,Ijagila Mark .,tel:+2348100001300,wulangaalfred@gmail.com,Wullanga Alfred,tel:+2347032853473,aidaticha@gmail.com,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,Reuben Aidaticha.,tel:+2348038213859,wullangaalfred@gmail.com,Wullanga Alfred.,tel:+2348029864318,wumi.ajayi@yahoo.com,Olawumi Monica Ajayi.,tel:+2348036173901,None,None,None
1,None,Monday Leasado.,tel:+2348038056048,None,Monday Leasado.,tel:+2348086625852,None,Monday Leasado.,tel:+2348065356507,None,None,None,None,None,None,Monday Leasado.,tel:+2348038056048,202,FUFORE LGA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,Hauwa Zoakah.,tel:+2347035678763,ijagilamark@gmail.com,Ijagila Mark .,tel:+2348100001300,wulangaalfred@gmail.com,Wullanga Alfred,tel:+2347032853473,aidaticha@gmail.com,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,Reuben Aidaticha.,tel:+2348038213859,wullangaalfred@gmail.com,Wullanga Alfred.,tel:+2348029864318,wumi.ajayi@yahoo.com,Olawumi Monica Ajayi.,tel:+2348036173901,None,None,None
2,None,Monday Leasado.,tel:+2348038056048,None,Monday Leasado.,tel:+2348086625852,None,Monday Leasado.,tel:+2348065356507,None,None,None,None,None,None,Monday Leasado.,tel:+2348086625852,202,FUFORE LGA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,Hauwa Zoakah.,tel:+2347035678763,ijagilamark@gmail.com,Ijagila Mark .,tel:+2348100001300,wulangaalfred@gmail.com,Wullanga Alfred,tel:+2347032853473,aidaticha@gmail.com,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,Reuben Aidaticha.,tel:+2348038213859,wullangaalfred@gmail.com,Wullanga Alfred.,tel:+2348029864318,wumi.ajayi@yahoo.com,Olawumi Monica Ajayi.,tel:+2348036173901,None,None,None
3,None,Monday Leasado.,tel:+2348038056048,None,Monday Leasado.,tel:+2348086625852,None,Monday Leasado.,tel:+2348065356507,None,None,None,None,None,None,Monday Leasado.,tel:+2348065356507,202,FUFORE LGA,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,hauwabata@yahoo.com,Hauwa Zoakah .,tel:+2348020938959,hauwabata@yahoo.com,Hauwa Zoakah.,tel:+2347035678763,ijagilamark@gmail.com,Ijagila Mark .,tel:+2348100001300,wulangaalfred@gmail.com,Wullanga Alfred,tel:+2347032853473,aidaticha@gmail.com,Reuben Aidaticha.,tel:+2348088481006,aidaticha@gmail.com,Reuben Aidaticha.,tel:+2348038213859,wullangaalfred@gmail.com,Wullanga Alfred.,tel:+2348029864318,wumi.ajayi@yahoo.com,Olawumi Monica Ajayi.,tel:+2348036173901,None,None,None
4,naatiwaziri69@gmail.com,Naaticha Waziri .,tel:+2348037675670,None,None,None,None,None,None,None,None,None,None,None,None,Naaticha Waziri .,tel:+234803767567

In [1102]:
#convert to xls
# and/or export as JSON
# Test Export as Excel 
filename = "IMAM_supervision.xlsx"
writer = pd.ExcelWriter(filename, engine='xlsxwriter')
final_df.to_excel(writer,'Sheet1')
writer.save()
writer.close()

In [90]:
imam_supervision['sitename'] = imam_supervision['siteid_x'].map(lambda x: Site.objects.get(siteid=x).sitename.strip() if Site.objects.filter(siteid=x) else "")

,name,urn,siteid_x,mail,lga_num,state_num_x,siteid_y,lganame1,lganame2,lganame3,...,snomail10,snomail11,snomail12,snomail13,snomail14,snomail15,snomail16,snomail17,snomail18,sitename
0,Munirat Makama.,tel:+2348165422728,2003110030,None,2003.0,20.0,2003.0,Assama'u Tukur.,Abdulkadir Muhammad Yasore.,,...,zayyanais@gmailcom,None,None,None,None,None,None,None,None,KANDAWA
1,Kwatam Dandy.,tel:+2348063455263,821110032,None,821.0,8.0,821.0,Hadiza Shettima.,,,...,None,None,None,None,None,None,None,None,None,CBN IDP Camp
2,Amina Musa T.,tel:+2348160109849,811110033,None,811.0,8.0,811.0,Mairo musa.,,,...,None,None,None,None,None,None,None,None,None,PHC Gwoza
3,HARUNA ABDULLAHI.,tel:+2348021010530,805110012,None,805.0,8.0,805.0,Aishatu Abubakar.,Aishatu Abubakar.,,...,None,None,None,None,None,None,None,None,None,GUR
4,Halima Muhammed.,tel:+2349076655644,821110031,None,821.0,8.0,821.0,Hadiza Shettima.,,,...,None,None,None,None,None,None,None,None,None,Teachers Village IDP Camp


In [ ]:
imam_supervision.head()

In [124]:
Site.objects.all()

<QuerySet [<Site: Site Bagega PHC>, <Site: Site Kasumka Comm Disp>, <Site: Site Kawaye Disp>, <Site: Site Makakari Disp>, <Site: Site Tungar Daji Disp (Anka)>, <Site: Site Tungar Kudaku >, <Site: Site Barayar Zaki Primary HC>, <Site: Site Bardi Disp>, <Site: Site Dutsin Dan Ajiya Disp>, <Site: Site Rafin Gero PHC>, <Site: Site Anka WCW Clinic>, <Site: Site Anka Psychiatric Hosp.>, <Site: Site Gargam Dispensary>, <Site: Site Galadunci Desp.>, <Site: Site inwala Disp>, <Site: Site Kadadabba Disp>, <Site: Site Abare Disp>, <Site: Site Anka GH>, <Site: Site Anka orphans and less previlaged Clinic>, <Site: Site Dareta Dispensary>, '...(remaining elements truncated)...']>

In [ ]:
# are any SiteIDs NaN ?

# print df.siteid.str.isdigit()
# df['siteid'].str.contains('^\d+$')

# SiteIDs in PostGres are BigInt thus strings are not allowed. 

# In load contact data for the SiteID, 
# we strip all following text after 9 or 10 digit number
# we convert all letter Os to zeros
# we delete all entries that have text before the siteID. 

In [ ]:
# are any SiteIDs NaN ?
df.query('siteid!=siteid')

In [ ]:
df2['siteid'] = pd.to_numeric(df2['siteid'], errors='coerce')
# No change

# Remove NaN from siteid

# If you don't specify a variable this will drop any rows including a NaN
df2 = df2.dropna(subset=['siteid'])
# No change

df2 = df2.query('siteid<3699990999')
# No change

In [ ]:
df2['siteid'] = df2['siteid'].astype(int)
# No change

In [ ]:
# What is the type of siteid
print type(df2['siteid'])
# This does not tell us anything about the variable

In [ ]:
df2.dtypes

In [ ]:
# do any SiteIDs include the letter o instead of number zero? 
foo = pd.DataFrame({'a' : [1,2,3,4], 'b' : ['hi', 'ooo', 'fat', 'cat']})
foo[foo['b'].str.contains('ooo')]  

# Regex & Data Cleaning
# https://trendct.org/2016/08/05/real-world-data-cleanup-with-python-and-pandas/

# df[df['siteid'].str.contains('ooo')]

# can only use .str accessor with string
#.str.contains('\D')

In [ ]:
# To replace
# data['result'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')


In [ ]:
# How many registrations in supervision ? 
len(supervision_df['siteid'])

# Many supervision staff did not record their post - should recode them all as supervisors
supervision_df['post'].str.upper().value_counts()
# str.upper() does not make permanent change. 

# I think there was a data entry error with Post because most people entered their post correctly
# also there should not be any HC, HW in the supervision cadres

In [ ]:
pd.unique(state_df.name.ravel())
# Ravel - Return the flattened underlying data as an ndarray